In [12]:
import numpy as np
import pandas as pd
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")


# Step 3: Now it's safe to modify
#movies['crew'] = movies['crew'].apply(fetch_director)


In [13]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [14]:
credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [15]:
credits.shape

(4803, 4)

In [16]:
movies.shape

(4803, 20)

In [17]:
movies = movies.merge(credits,on='title')
movies.head()
movies.shape

(4809, 23)

In [18]:
#budget is not a criteria to recommend a movie
#if we observe language almost 95% is english so no need
movies['original_language'].value_counts()

original_language
en    4510
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
ko      12
cn      12
ru      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
ta       2
cs       2
ro       2
id       2
ar       2
vi       1
sl       1
ps       1
no       1
ky       1
hu       1
pl       1
af       1
nb       1
tr       1
is       1
xx       1
te       1
el       1
Name: count, dtype: int64

In [19]:
#we need to extract only necessary columns for out prediction
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [20]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [21]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [22]:
#in crew i only consider direcor
#in cast i only consider top 3 actors
movies.dropna(inplace=True)
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [23]:
movies.iloc[0].genres
#we need to convert ths dictionary to something lik ['action','adventure','fantasy',Science ficiton']

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [24]:
import ast
def convert(obj):
  l=[]
  for i in ast.literal_eval(obj):

    l.append(i['name'])
  return l

In [25]:
# to get list of genres of all movies
movies['genres'] = movies['genres'].apply(convert)
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [26]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [27]:
import ast

def convertt(obj):
    # If it's already a list, return the first 3 names directly
    if isinstance(obj, list):
        return obj[:3]
    
    l = []
    cnt = 0
    try:
        for i in ast.literal_eval(obj):
            if cnt < 3:
                l.append(i['name'])
                cnt += 1
            else:
                break
    except:
        pass  # Skip bad data
    return l


In [28]:
#actually i have converted to list when i run it again then im getting error so i will
#comment this 
movies['cast'] = movies['cast'].apply(convertt)
#getting only 3 names from cast
movies['cast']

0        [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1           [Johnny Depp, Orlando Bloom, Keira Knightley]
2            [Daniel Craig, Christoph Waltz, Léa Seydoux]
3            [Christian Bale, Michael Caine, Gary Oldman]
4          [Taylor Kitsch, Lynn Collins, Samantha Morton]
                              ...                        
4804    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4805         [Edward Burns, Kerry Bishé, Marsha Dietlein]
4806           [Eric Mabius, Kristin Booth, Crystal Lowe]
4807            [Daniel Henney, Eliza Coupe, Bill Paxton]
4808    [Drew Barrymore, Brian Herzlinger, Corey Feldman]
Name: cast, Length: 4806, dtype: object

In [29]:
import ast
def fetch_director(obj):
  l=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':

      l.append(i['name'])
  return l

In [30]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [31]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [32]:

#converting overview contents to list
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [33]:
#u can observe overview is changed to list 
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [34]:
#remove spaces between singl eword bcoz if we keep space it will treat like 2 diff entiites but ex= Sam Worthington is name of one person
#but it treate as sam diff and worthington diff in order to overcome this isuue lets combine them(apply transformation)
movies['genres'] =  movies['genres'].apply(lambda x:[i.replace(" ","") for i in x]) 


In [35]:
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x]) 
movies['cast']= movies['cast'].apply(lambda x:[i.replace(" ","") for i in x]) 
movies['crew']= movies['crew'].apply(lambda x:[i.replace(" ","") for i in x]) 

In [36]:
movies.head()
#observe spaces removed

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [37]:
movies['tag'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] +movies['crew']

In [38]:
movies.head(2)
#adding a new ag col which i s combo of 5 columns

,movie_id,title,overview,genres,keywords,cast,crew,tag
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."


In [39]:
new_df = movies[['movie_id', 'title', 'tag']]
new_df['tag'] = new_df['tag'].apply(lambda x: " ".join(x))

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_124\3862497520.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tag'] = new_df['tag'].apply(lambda x: " ".join(x))


In [40]:
new_df.head(2)

,movie_id,title,tag
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."


In [41]:
new_df['tag'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [42]:
new_df['tag'] = new_df['tag'].apply(lambda x:x.lower())

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_124\1436014886.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tag'] = new_df['tag'].apply(lambda x:x.lower())


In [43]:
new_df.head(2)

,movie_id,title,tag
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."


In [44]:
#vectorisation
# vectorization is crucial because it allows the system to convert textual data, like movie names, cast, descriptions, or reviews,
#into a numerical representation that can be processed by algorithms. Machine learning and recommendation systems typically cannot understand
#raw text directly, so we need to convert this text into a format that can be mathematically analyzed and compared.
# vectors (arrays of numbers) that maintain the semantic meaning of the text.
#if you convert two movies' descriptions or cast lists into vectors, you can compute the similarity between them. If the vectors are
#close in value, it means the movies are similar in content or attributes, and thus, one movie could be recommended based on the other.

In [45]:
#most common tech we are going to use is bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')
#wt are the 5000 most freq words

In [46]:
vectors = cv.fit_transform(new_df['tag']).toarray()

In [47]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [48]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [49]:
len(cv.get_feature_names_out())

5000

In [50]:
new_df['tag'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [51]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [52]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [59]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)    

In [54]:
# in those feature names we have similar names like 'love' ,'loved', 'loving' 
# so we have to consider all of them as only 'love'
ps.stem('loveing')
ps.stem('loved')

'love'

In [70]:
ps.stem('arrested')

'arrest'

In [60]:
#lets check stem of tag[0]
#observe marine is changed to marin
stem('in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron')

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [61]:
#so we have to apply stem to all tags of data
new_df['tag'] = new_df['tag'].apply(stem)

0       in the 22nd century, a parapleg marin is dispa...
1       captain barbossa, long believ to be dead, ha c...
2       a cryptic messag from bond’ past send him on a...
3       follow the death of district attorney harvey d...
4       john carter is a war-weary, former militari ca...
                              ...                        
4804    el mariachi just want to play hi guitar and ca...
4805    a newlyw couple' honeymoon is upend by the arr...
4806    "signed, sealed, delivered" introduc a dedic q...
4807    when ambiti new york attorney sam is sent to s...
4808    ever sinc the second grade when he first saw h...
Name: tag, Length: 4806, dtype: object

In [68]:
cv.get_feature_names_out()[20:400]

array(['1970s', '1971', '1974', '1976', '1980', '1980s', '1985', '1990s',
       '1999', '19th', '19thcentury', '20', '200', '2003', '2009', '20th',
       '24', '25', '30', '300', '3d', '40', '50', '500', '60', '60s',
       '70', 'aaron', 'aaroneckhart', 'abandoned', 'abducted',
       'abigailbreslin', 'abilities', 'ability', 'able', 'aboard',
       'abuse', 'abusive', 'academic', 'academy', 'accept', 'accepted',
       'accepts', 'access', 'accident', 'accidental', 'accidentally',
       'accompanied', 'accomplish', 'account', 'accountant', 'accused',
       'ace', 'achieve', 'act', 'acting', 'action', 'actionhero',
       'actions', 'activist', 'activities', 'activity', 'actor', 'actors',
       'actress', 'acts', 'actual', 'actually', 'adam', 'adams',
       'adamsandler', 'adamshankman', 'adaptation', 'adapted', 'addict',
       'addicted', 'addiction', 'adolescence', 'adopt', 'adopted',
       'adoption', 'adopts', 'adrienbrody', 'adult', 'adultery',
       'adulthood', 'adult

In [73]:
#here we have 4806 movies i.e rows/vectors   and 5000 columns for each row
#Cosine cares about the direction of vectors (not their magnitude), which makes it perfect for comparing how similar the
#content is, not how long the description is.
# so we use cosine dis/similarity not euclidean dis.  euclid dis in not reliable measure as this have high dimensionality
#Movies with similar themes, genres, or keywords to be treated as similar
#Even if one has 10 tags and the other has 50

from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(vectors)
#the meaning here is we are calculating dis for 1st moviw to all remaining movies like 2nd movie to dis btw remainig other like that 
# 4086 movies distanes with 480 movies

array([[1.        , 0.08964215, 0.06071767, ..., 0.02519763, 0.0277885 ,
        0.        ],
       [0.08964215, 1.        , 0.06350006, ..., 0.02635231, 0.        ,
        0.        ],
       [0.06071767, 0.06350006, 1.        , ..., 0.02677398, 0.        ,
        0.        ],
       ...,
       [0.02519763, 0.02635231, 0.02677398, ..., 1.        , 0.07352146,
        0.04774099],
       [0.0277885 , 0.        , 0.        , ..., 0.07352146, 1.        ,
        0.05264981],
       [0.        , 0.        , 0.        , ..., 0.04774099, 0.05264981,
        1.        ]])

In [75]:
cosine_similarity(vectors).shape

(4806, 4806)

In [78]:
similar = cosine_similarity(vectors)

In [79]:
similar[0]
#diagonal ele are 1 bcoz same movie 

array([1.        , 0.08964215, 0.06071767, ..., 0.02519763, 0.0277885 ,
       0.        ])

In [90]:
sorted(list(enumerate(similar[0])), reverse = True)
#we dont want this sorting based on index

[(4805, 0.0),
 (4804, 0.027788500718836418),
 (4803, 0.02519763153394848),
 (4802, 0.05345224838248488),
 (4801, 0.02492223931396134),
 (4800, 0.0),
 (4799, 0.05884898863364997),
 (4798, 0.02129588549999799),
 (4797, 0.0),
 (4796, 0.0),
 (4795, 0.0),
 (4794, 0.0),
 (4793, 0.059761430466719674),
 (4792, 0.0),
 (4791, 0.0),
 (4790, 0.03194382824999699),
 (4789, 0.023002185311411804),
 (4788, 0.0),
 (4787, 0.02258769757263128),
 (4786, 0.0),
 (4785, 0.0),
 (4784, 0.04517539514526256),
 (4783, 0.0),
 (4782, 0.027066598098038335),
 (4781, 0.06388765649999398),
 (4780, 0.023002185311411804),
 (4779, 0.0),
 (4778, 0.0),
 (4777, 0.05345224838248488),
 (4776, 0.0),
 (4775, 0.03253000243161777),
 (4774, 0.0),
 (4773, 0.029880715233359837),
 (4772, 0.026398183867422733),
 (4771, 0.042257712736425826),
 (4770, 0.0),
 (4769, 0.0),
 (4768, 0.0),
 (4767, 0.03877833671647406),
 (4766, 0.018227065414412227),
 (4765, 0.0),
 (4764, 0.0),
 (4763, 0.0),
 (4762, 0.0),
 (4761, 0.03194382824999699),
 (4760, 0

In [94]:
#i want to recommend top 5 movies similar to given movie name
# step 1 : to fetch index of that movie 
#step 2 : get didtances of all movies from that index movie using simalar df
#step 3: sort the distances in descending and fetch staring 5 
    #(why descending means as we are using cosine similarity the vale 1.0 means closer value 0 means not at all similar to out given movie
    #if we use cosine dis then we can go with ascending order itself
#step 4 : but if we sort we lose indices so lets USE ENUMERATE FUNCTION if we directly use sort with enumerate we get 
#sorting based on index but we want to be based on distance so use lamda func also
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similar[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse = True,key =lambda x: x[1])[1:6]

    for i in movies_list:
        
        
        print(new_df.iloc[i[0]].title)
    

    
    

In [95]:
recommend('Avatar')

Titan A.E.
Small Soldiers
Ender's Game
Aliens vs Predator: Requiem
Independence Day


In [101]:
#this file created in folder 
import pickle
pickle.dump(new_df.to_dict(), open('movie_dict.pkl', 'wb'))

In [102]:
new_df['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [105]:
pickle.dump(similar, open('similar.pkl', 'wb'))